<a href="https://colab.research.google.com/github/zahra-eslamian/artificial_intelligence_A_to_Z/blob/main/Deep_Q_Learning_for_Lunar_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch # we will build our AI using pytorch
import torch.nn as nn # importing Neural Network module
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd # for Stochastic Gradient Descent
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):
  # state_size: the number of inputs into the space
  def __init__(self, state_size, action_size, seed):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    # the first fully connected layer connecting the input layer to the second layer
    self.fc1 = nn.Linear(state_size, 64)
    # the second fully connected layer connecting the second layer to the third layer / 64 is obtained via trial & error as the optimal number of neurons
    self.fc2 = nn.Linear(64, 64)
    # the second fully connected layer connecting the third layer to the output layer
    self.fc3 = nn.Linear(64, action_size)

  # defining a method to forward propagate the signal from the input layer to the output layer through our 2 fully connected layers
  def forward(self, state):
    # F is the alias we defined for torch.nn.functional, and from that we call the relu function which is in fact the rectifier function
    x = F.relu(self.fc1(state))
    x = F.relu(self.fc2(x))
    return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [4]:
import gymnasium as gym
env = gym.make("LunarLander-v3")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State Shape: ', state_shape)
print('State Size: ', state_size)
print('Number of Actions: ', number_actions)

State Shape:  (8,)
State Size:  8
Number of Actions:  4


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4 # 0.0005, it's specified with trial & error

minibatch_size = 100 # number of observations used in one step of training to update the model parameters, 100 is selected based on common practice

discount_factor = 0.99 # also knwon as gamma, it represents the present value of future rewards.
# Close to zero makes the agent shortsighted by only considering the current rewards,
# close to one makes the agent consider the future rewards in its accumulation of the total reward - > best performance

replay_buffer_size = int(1e5) # (100000) the size of the AI memory, how many experiences including (state, action, reward, next state, done or not) in the agent's memory

interpolation_parameter = 1e-3 # (0.001) it will be used in the sub update of the target network, the value is chosen with trial & error


### Implementing Experience Replay

In [6]:
class ReplayMemory(object):

  def __init__(self, capacity):
    # if you want to run this code outside of Colab, if you have GPU you can use it to make the training faster
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = [] # (state, action, reward, next state, done or not)

# add succefully an experience event into the memory buffer, making sure that memory buffer does not exceed its capacity
  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0] # remove the oldest experience

# randomly select batch of experiences from the memory buffer
  def  sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    # extract and stack all the elements of an experience (state, action, reward, next state, done)
    # with the vstack function from numpy, we stack all the states of all the batch_size experinces selected randomly
    # then, we need to convert those stacks into PyTorch Tensors, becuase we will do the training with PyTorch
    # then, we should make sure that the data type of the states (which have become PyTorch Tensors) is float as it is expected by torch library for training
    # at the end, we add the stacks to our device
    states = torch.from_numpy(np.vstack(list(e[0] for e in experiences if e is not None))).float().to(self.device)
    actions = torch.from_numpy(np.vstack(list(e[1] for e in experiences if e is not None))).long().to(self.device)
    rewards = torch.from_numpy(np.vstack(list(e[2] for e in experiences if e is not None))).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(list(e[3] for e in experiences if e is not None))).float().to(self.device)
    # as Done is a Boolean, we should first convert it to Boolean then to float Tensors
    dones = torch.from_numpy(np.vstack(list(e[4] for e in experiences if e is not None)).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [7]:
class Agent():

  def __init__(self, state_size, action_size, seed):
    self.state_size = state_size
    self.action_size = action_size
    self.seed = random.seed(seed)
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.local_qnetwork = Network(state_size, action_size, seed).to(self.device) # local_qnetwork will select the actions
    self.target_qnetwork = Network(state_size, action_size, seed).to(self.device) #target_qnetwork will calculate the target Q values that will be used to train the local Q Network
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.time_step = 0

  # to store experiences and decide when to learn from them
  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.time_step = (self.time_step + 1) % 4 # we want to learn every 4 steps, so we reset time_step every 4 steps
    if self.time_step == 0:
      if len(self.memory.memory) > minibatch_size: # in self.memory.memory, self.memory is the instance of ReplayMemory class, and the second memory if the attribute of that instance
        experiences = self.memory.sample(minibatch_size)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    # first we make sure that state is a Torch Tensor
    # we need to add an extra dimension to our state vector corresponding to the batch, to specify which batch this state belongs to
    # we add that extra dimention to the index 0, first dimension: unsqueeze(0)
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    # we first set our local Q network to evaluation mode
    self.local_qnetwork.eval()
    # we make sure that we are not in the training mode but in inference mode meaning that the model is making predictions on new, unseen data
    with torch.no_grad(): # make sure that any gradient computation is disabled
      # forward pass the state to the output layer through our 2 fully connected layers to get the Q values corresponding to each action
      action_values = self.local_qnetwork(state)
    # now we go back to the training mode
    self.local_qnetwork.train()
    # epsilon-greedy action selection policy
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)


### Initializing the DQN agent

In [8]:
agent = Agent(state_size, number_actions, seed=0)

### Training the DQN agent

In [9]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset() # with this _, we discard all the other arguments that a fucntion returns and we don't need them
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)

  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -167.80
Episode 200	Average Score: -130.80
Episode 300	Average Score: -66.09
Episode 400	Average Score: -30.72
Episode 500	Average Score: -19.93
Episode 600	Average Score: 23.44
Episode 700	Average Score: 102.88
Episode 800	Average Score: 150.66
Episode 900	Average Score: 150.29
Episode 973	Average Score: 200.30
Environment solved in 873 episodes!	Average Score: 200.30


## Part 3 - Visualizing the results

In [10]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()